# Implementación de Bagging

En este Notebook llevaremos acabo un ejemplo de implementación de Bagging, para facilitar el entendimiento del funcionamiento de los modelos. 

Se utilizará como modelo base el `DecisionTreeRegressor` de Sci-Kit Learn para el problema de Predicción de Precios de Viviendas en Boston. 

En este notebook usted irá implementando paso a paso los elementos que componen el algoritmo. 



<table class="tfo-notebook-buttons" align="center">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jsebastianquiroga/analitica_ia_puj/blob/main/Analitica_2/02_bagging.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jsebastianquiroga/analitica_ia_puj/blob/main/Analitica_2/02_bagging.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /></a>
  </td>
  
</table>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn import datasets
from tqdm import tqdm
import pandas as pd
import numpy as np

## Carga de Datos

Utilizaremos los Datasets de Sci-Kit Learn para cargar la información de precios de vivienda.

In [4]:
from sklearn.datasets import fetch_openml
housing = fetch_openml(name="house_prices", as_frame=True)
boston_housing = housing

# boston_housing = load_boston()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
boston_housing.keys() # Elementos con los que vienen los datos

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

Estos datos vienen como arrays de NumPy y sin nombres de campos. A cotninuación los convertiremos en DataFrames y asignaremos los nombres a nuestras variables explicativas.

In [6]:
X = pd.DataFrame(boston_housing['data'], columns = boston_housing['feature_names'])
y = pd.DataFrame(boston_housing['target'])

In [7]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,None,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,None,0,12,2008,WD,Normal


In [8]:
y.head()

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


## Pre-Procesamiento de Datos

A continuación dividiremos la información en Train y Test, y realizaremos un escalamiento de datos.

Para la división en train-test, utilice la función de [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
### Utilice train_test_split para dividir los datos en entrenamiento y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
###

In [ ]:
X_train.head()

Se sugiere realizar un MinMax scaler como procesamiento inicial de las features. Usted puede probar con otros procesamientos. 

Para esto, utilice [`sklearn.preprocessing.MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html). 

In [ ]:
###

scaler = StandardScaler()
scaler.fit(X_train)
X_train.values[:] = scaler.transform(X_train)
X_test.values[:] = scaler.transform(X_test)

###

In [ ]:
X_train

Con el procesamiento anterior, es posible comenzar a programar los modelos que vamos a utilizar.

# Implementación de Bagging

Recordemos que Bagging consiste de tres elementos: 
- Muestreo aleatorio con reemplazamiento de los datos de entrenamiento (muestras bootstrap)

- Entrenamiento del mismo modelo en cada una de las diferentes $B$ muestras, 
$$\Big\{\hat{f}_b(x) \Big\}_{b = 1, \dots, B}$$

- Combinación de predicciones a través de un promedio simple, 

$$\hat{f}_{bagging} (x) = \frac{1}{B} \sum_{b = 1}^B \hat{f}_b(x)$$

Vamos a crear una clase que nos permita estimar un conjunto de árboles de regresión ($\hat{f}_b(x)$) y lo utilizaremos para predecir. 

## 1. Boostrap Sampling

El primer elemento de la función es quizás el más importante del *Bagging*, pues corresponde al muestreo aleatorio con reemplazamiento. 


Utilice el métoo de [`pd.DataFrame.sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) para tomar una muestra aleatoria con reemplazamiento **de las filas**.

In [ ]:
def _sample_features(self, X, y, random_state = None):
  """
  Generación de [una] muestra Bootstrap para entrenar un modelo de árbol de regresión.

  Params
  --------
    X (pandas.DataFrame):
      Un dataframe que contiene los features de entrenamiento

    y (pandas.DataFrame):
      Un dataframe que contiene el target para el entrenamiento

    random_state (int):
      [Default None] Valor de semilla. Utilizar únicamente para pruebas.


  Returns
  --------
    sampled_x (pandas.DataFrame):
      Un dataframe que contiene una muestra bootstrap de los datos originales

    sampled_y (pandas.DataFrame):
      Un dataframe que contiene una la variable dependiente de cada observación.
  """
  ###

  sampled_x = X.sample(replace = True, frac = 1, axis = 0, random_state = random_state)
  sampled_y = y.loc[sampled_x.index] # Asigne a cada X su Y.

  ###

  return sampled_x, sampled_y

Si la función fue programada correctamente, entonces el resultado de la siguiente celda debería ser:

|         |         0 |
|:--------|----------:|
| CRIM    | -22.3237  |
| ZN      |  -3.87571 |
| INDUS   |  -6.34016 |
| CHAS    |  -3.83336 |
| NOX     | -35.7036  |
| RM      | -21.231   |
| AGE     | -46.6513  |
| DIS     |  22.7396  |
| RAD     | -21.3265  |
| TAX     | -32.3898  |
| PTRATIO |  37.1405  |
| B       | -10.3039  |
| LSTAT   |  10.5585  |

In [ ]:
X_sampled_fn, y_sampled_fn = _sample_features(None, X_train, y_train, random_state = 42)
X_sampled_fn.sum()

## Entrenamiento de un Árbol

La idea de Bagging es entrenar el mismo modelo en diferentes muestras de la base de entrenamiento original. Por esto, deberemos crear una función que genere la muestra *Bootstrap* y entrene el árbol de regresión en esta. 

Utilice [`sklearn.tree.DecisionTreeRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor.predict) para su estimador base (su weak learner).


In [ ]:
def train_single_tree(self, X, y):
  """
  Entrena un árbol de regresión en una muestra bootstrap de los datos originales.

   Params
  --------
    X (pandas.DataFrame):
      Un dataframe que contiene los features de entrenamiento

    y (pandas.DataFrame):
      Un dataframe que contiene el target para el entrenamiento

  Returns
  --------
    tree (sklearn.tree.DecisionTreeRegressor):
      Un modelo de árbol de regresión ajustado en una muestra 
      bootstrap de los datos de entrenamiento.

  """
  ###

  # 1. Utilice la función _sample_features para generar una muestra
  sampled_X, sampled_y = self._sample_features(X, y) 

  # 2. Entrene el modelo de regresión. Fije el parámetro max_features = self.col_prop
  tree = DecisionTreeRegressor(max_features = self.col_prop)

  # 3. Utilice el método .fit() para ajustar el modelo
  tree.fit(sampled_X, sampled_y)

  ###
  return tree

## Todo Junto

En la siguiente celda se tiene el código que nos permitirá entrenar en paralelo un número arbitrario de árboles. Esta clase utilizará las funciones definidas anteriormente. 

La función que se paraleliza es `train_single_tree` y la función de predicción `predict_multiple`.

In [ ]:
class BaggingRegressor:

  def __init__(self, n_trees = 200, col_prop = 0.6, n_jobs = 3):
    self.n_trees = n_trees
    self.col_prop = col_prop
  
    self.n_jobs = n_jobs

    self.model_list = None
    self.X = None
    self.y = None


  def parallel_fit_training(self, X, y):
    """
    Realiza un entrenamiento en paralelo de la cantidad de árboles especiicados en self.n_trees
    """
    self.model_list = Parallel(n_jobs=self.n_jobs, 
                               timeout=99999)(delayed(self.train_single_tree)(X, y) for _ in tqdm(range(self.n_trees)))


  def predict_multiple(self, X):
    """
    Realiza la predicción de cada elemento del ensamble. 

    Params
    -------
      X (pd.DataFrame):
        DataFrame que cotiene los features para predecir

    Returns
    -------
      prediction_array (numpy.array):
        Un array que contiene todas las predicciones de cada 
        elemento del ensamble.

    """

    prediction_list = Parallel(n_jobs=self.n_jobs, timeout=99999)(delayed(model.predict)((X[model.feature_names_in_])) for model in tqdm(self.model_list))
    prediction_array = np.array(prediction_list)
    return prediction_array

  def predict(self, X, reduce = True):
    """
    Método para realizar las predicciones. 

    Params
    --------
      X (pd.DataFrame):
        DataFrame que cotiene los features para predecir
      
      reduce (bool):
        [Default True] Indica si se desea que la predicción sea el promedio del 
        ensamble. Si reduce = False, entonces se devuelve la predicción
        de cada modelo individual.
    """
    prediction = self.predict_multiple(X)

    if reduce:
      return prediction.mean(axis = 0)
    else:
      return prediction.T

  def fit(self, X, y):
    """
    Ajusta el modelo.

    Params
    --------
      X (pd.DataFrame):
        Datos de entrenamiento (features)
      y (pd.DataFrame):
        Target para los datos de entrenamiento

    """
    self.parallel_fit_training(X, y)

En esta clase añadimos los métodos que creamos anteriormente.

In [ ]:
BaggingRegressor._sample_features = _sample_features

In [ ]:
BaggingRegressor.train_single_tree = train_single_tree

## Bagging Default & Random Forest

Como se habló en clase, el Random Forest es un modelo de Bagging, que busca disminuir la correlación entre los modelos al restringir las features a las que tiene acceso cada split. De esta forma, los árboles entrenados son diferentes unos de otros. 

Si en nuestro `BaggingRegressor` especificamos un `col_prop = 1`, estaremos entrenado un modelo Bagging estándar (default). Si `col_prop < 1`, se estará entrenando un `Random Forest`. 

A continuación, entrene un modelo Bagging con `col_prop = 1` y otro con `col_prop < 1`. Ambos modelos deben tener `n_trees = 1000`. 

In [ ]:
### Entrenamiento de BaggingEnsemble

BaggingEnsemble = BaggingRegressor(n_trees = 1000, col_prop = 1)
BaggingEnsemble.fit(X_train, y_train)

###

In [ ]:
### Entrenamiento de Random Forest

RandomForest = BaggingRegressor(n_trees = 1000) # el col prop es un hiperparámetro que toca tunear
RandomForest.fit(X_train, y_train)

###


## Desempeño del Modelo



## Evolución del Modelo

Anteriormente observamos cómo el `RandomForest` tuvo un mejor desempeño que el `BaggingEnsemble`. 

En esta sección revisaremos cómo evolucionan las métricas de desempeño del modelo a medida que se incluyen más `weak_learners` y si tienen algo que ver con la diferencia de los resultados de evaluación.


Realizaremos las predicciones nuevamente, pero esta vez se fijará el parámetro `reduce = False`. De esta forma, no obtendremos una predicción final, sino las predicciones que hicieron cada uno de los 1000 árboles. 

In [ ]:
### Predicciones de Bagging Ensemble

y_pred_test_bag =  BaggingEnsemble.predict(X_test, reduce = False)

###

In [ ]:
### Predicciones en Random Forest

y_pred_test_rf =  RandomForest.predict(X_test, reduce = False)

###

Una vez tenemos las predicciones, queremos ver cómo nos hubiera dado el error del modelo si únicamente hubieramos utilizado los primeros `m` árboles. 

In [ ]:
def mse_first_m_models(y_true, predictions, m):
  """
  Calcula el MSE de cada modelo utilizando únicamente las primeras m predicciones.
  """

  ### 
  # 1. Seleccione las primeras m predicciones y calcule el promedio
  mth_avg_pred = predictions[:, :m].mean(axis = 1)

  # 2. Calcule el MSE
  mse = mean_squared_error(y_true, mth_avg_pred)

  ###
  return mse

In [ ]:
n_trees = np.arange(1, 1000)

In [ ]:
mse_list_rf = [mse_first_m_models(y_test, y_pred_test_rf, m) for m in n_trees]
mse_list_bag = [mse_first_m_models(y_test, y_pred_test_bag, m) for m in n_trees]

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(n_trees, mse_list_rf, label = 'Bagging (RF) MSE')
plt.plot(n_trees, mse_list_bag, label = 'Bagging MSE')
plt.legend()
plt.xlabel('Number of Trees')
plt.ylabel('MSE')
plt.title('MSE vs Número de Árboles')
plt.show()

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(n_trees, mse_list_rf, label = 'Bagging (RF) MSE')
plt.plot(n_trees, mse_list_bag, label = 'Bagging MSE')
plt.legend()
plt.xlabel('Number of Trees')
plt.ylabel('MSE')
plt.title('MSE vs Número de Árboles')
plt.xlim(0, 100)
plt.show()